# AutoHDR Flow-Residual CalibGuard Notebook

Self-contained Kaggle notebook for exploratory dimension-conditioned residual flow correction.

Outputs under `/kaggle/working/autohdr_flow_residual/`:
- `submission_flow_residual_dim_v1_<timestamp>.zip`
- `submission_flow_residual_dim_base_<timestamp>.zip`
- `run_summary_<timestamp>.json`


In [ ]:
import os
import json
import time
import zipfile
import random
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from collections import Counter, defaultdict

import cv2
import numpy as np


SEED = int(os.getenv("SEED", "42"))
random.seed(SEED)
np.random.seed(SEED)

MIN_SUPPORT = int(os.getenv("MIN_SUPPORT", "100"))
TRAIN_SAMPLE_PER_DIM = int(os.getenv("TRAIN_SAMPLE_PER_DIM", "400"))
HOLDOUT_RATIO = float(os.getenv("HOLDOUT_RATIO", "0.2"))
FLOW_MAX_MAG = float(os.getenv("FLOW_MAX_MAG", "40.0"))
LAMBDA_GRID = [float(x) for x in os.getenv("LAMBDA_GRID", "0,0.02,0.04,0.06,0.08,0.10").split(",") if x.strip()]
ACCEPT_MIN_PROXY_GAIN = float(os.getenv("ACCEPT_MIN_PROXY_GAIN", "0.002"))
PROFILE = os.getenv("PROFILE", "balanced")
JPEG_QUALITY = int(os.getenv("JPEG_QUALITY", "95"))

FLOW_WORK_SCALE = 0.25
FLOW_SMOOTH_SIGMA = 1.2
FLOW_STABILITY_VAR_MAX = 3.5
BORDER_RATIO_MAX = 0.002
WARP_RISK_MAX = 0.35

WORK_ROOT = Path("/kaggle/working/autohdr_flow_residual")
CANDIDATE_DIR = WORK_ROOT / "candidate"
BASE_DIR = WORK_ROOT / "base"
WORK_ROOT.mkdir(parents=True, exist_ok=True)
CANDIDATE_DIR.mkdir(parents=True, exist_ok=True)
BASE_DIR.mkdir(parents=True, exist_ok=True)

DEFAULT_DIM_TABLE = {
    "global_fallback": {"k1": -0.17, "k2": 0.35},
    "parent_classes": {
        "standard": {"primary": {"k1": -0.17, "k2": 0.35}, "safe": {"k1": -0.165, "k2": 0.34}},
        "near_standard_tall": {"primary": {"k1": -0.17, "k2": 0.35}, "safe": {"k1": -0.165, "k2": 0.34}},
        "near_standard_short": {"primary": {"k1": -0.17, "k2": 0.35}, "safe": {"k1": -0.16, "k2": 0.33}},
        "moderate_crop": {"primary": {"k1": -0.08, "k2": 0.15}, "safe": {"k1": -0.055, "k2": 0.1}},
        "heavy_crop": {"primary": {"k1": -0.01, "k2": 0.01}, "safe": {"k1": -0.002, "k2": 0.005}},
        "portrait_standard": {"primary": {"k1": -0.02, "k2": 0.02}, "safe": {"k1": -0.008, "k2": 0.015}},
        "portrait_cropped": {"primary": {"k1": -0.005, "k2": 0.005}, "safe": {"k1": 0.0, "k2": 0.0}},
    },
    "dimensions": {},
}


@dataclass
class DimModel:
    dim_key: str
    support: int
    parent_class: str
    k1: float
    k2: float
    model_type: str
    lambda_value: float
    accepted: bool
    reason: str
    proxy_gain: float
    base_loss: float
    best_loss: float
    border_ratio: float
    flow_var: float
    flow_p99: float
    flow_clip_max: float
    warp_risk: float
    flow_map: np.ndarray | None


def classify_parent(height: int, width: int) -> str:
    is_portrait = height > width
    short_edge = width if is_portrait else height
    if is_portrait:
        return "portrait_standard" if short_edge == 1367 else "portrait_cropped"
    if short_edge == 1367:
        return "standard"
    if short_edge in (1368, 1369, 1370, 1371):
        return "near_standard_tall"
    if short_edge in (1365, 1366):
        return "near_standard_short"
    if 1360 <= short_edge <= 1364:
        return "moderate_crop"
    return "heavy_crop"


def choose_coeffs(table: dict, dim_key: str, parent_class: str, profile: str):
    dimensions = table.get("dimensions", {})
    parent_classes = table.get("parent_classes", {})
    global_fallback = table.get("global_fallback", {"k1": -0.17, "k2": 0.35})
    entry = dimensions.get(dim_key)
    parent = parent_classes.get(parent_class, {})

    def fallback_primary():
        node = parent.get("primary")
        if isinstance(node, dict) and "k1" in node and "k2" in node:
            return float(node["k1"]), float(node["k2"]), "parent_primary"
        return float(global_fallback["k1"]), float(global_fallback["k2"]), "global_fallback"

    def fallback_safe():
        node = parent.get("safe")
        if isinstance(node, dict) and "k1" in node and "k2" in node:
            return float(node["k1"]), float(node["k2"]), "parent_safe"
        return float(global_fallback["k1"]), float(global_fallback["k2"]), "global_fallback"

    if not isinstance(entry, dict):
        if profile == "safe":
            return fallback_safe()
        return fallback_primary()

    support = int(entry.get("support", 0))
    force_safe = bool((entry.get("guardrails") or {}).get("force_safe", False))
    primary = entry.get("primary") or {}
    safe = entry.get("safe") or {}

    if not isinstance(primary, dict) or "k1" not in primary or "k2" not in primary:
        p_k1, p_k2, _ = fallback_primary()
    else:
        p_k1, p_k2 = float(primary["k1"]), float(primary["k2"])

    if not isinstance(safe, dict) or "k1" not in safe or "k2" not in safe:
        s_k1, s_k2, _ = fallback_safe()
    else:
        s_k1, s_k2 = float(safe["k1"]), float(safe["k2"])

    if profile == "safe":
        return s_k1, s_k2, "dim_safe"
    if profile == "balanced":
        if force_safe:
            return s_k1, s_k2, "dim_safe_guardrail"
        return p_k1, p_k2, "dim_primary"
    if force_safe or support < 100:
        return s_k1, s_k2, "dim_safe"
    return p_k1, p_k2, "dim_primary"


def choose_model_type(table: dict, dim_key: str) -> str:
    entry = (table.get("dimensions") or {}).get(dim_key, {})
    if isinstance(entry, dict):
        model = entry.get("model") or {}
        if isinstance(model, dict):
            kind = str(model.get("type", "")).strip().lower()
            if kind in {"brown", "rational"}:
                return kind
    defaults = table.get("model_defaults") or {}
    if isinstance(defaults, dict):
        kind = str(defaults.get("type", "")).strip().lower()
        if kind in {"brown", "rational"}:
            return kind
    return "brown"


def apply_base_undistortion(image: np.ndarray, k1: float, k2: float, model_type: str) -> np.ndarray:
    h, w = image.shape[:2]
    focal = float(np.sqrt(w * w + h * h))
    camera = np.array([[focal, 0.0, w / 2.0], [0.0, focal, h / 2.0], [0.0, 0.0, 1.0]], dtype=np.float64)
    if model_type == "rational":
        dist = np.array([k1, k2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], dtype=np.float64)
    else:
        dist = np.array([k1, k2, 0.0, 0.0, 0.0], dtype=np.float64)
    new_camera, _ = cv2.getOptimalNewCameraMatrix(camera, dist, (w, h), 0.0, (w, h))
    corrected = cv2.undistort(image, camera, dist, None, new_camera)
    if corrected.shape[:2] != (h, w):
        corrected = cv2.resize(corrected, (w, h), interpolation=cv2.INTER_LANCZOS4)
    return corrected


def compute_dense_flow(src_bgr: np.ndarray, tgt_bgr: np.ndarray) -> np.ndarray:
    src = cv2.cvtColor(src_bgr, cv2.COLOR_BGR2GRAY)
    tgt = cv2.cvtColor(tgt_bgr, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(
        src, tgt, None,
        pyr_scale=0.5, levels=4, winsize=17,
        iterations=4, poly_n=7, poly_sigma=1.4,
        flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN,
    )
    return flow.astype(np.float32)


def clip_flow_magnitude(flow: np.ndarray, clip_max: float) -> np.ndarray:
    if clip_max <= 0:
        return np.zeros_like(flow)
    mag = np.sqrt(flow[..., 0] ** 2 + flow[..., 1] ** 2)
    safe_mag = np.maximum(mag, 1e-6)
    scale = np.minimum(1.0, clip_max / safe_mag)
    return (flow * scale[..., None]).astype(np.float32)


def aggregate_residual_flow(flow_stack: list[np.ndarray], flow_max_mag: float):
    stack = np.stack(flow_stack, axis=0).astype(np.float32)
    median_flow = np.median(stack, axis=0).astype(np.float32)
    med_mag = np.sqrt(median_flow[..., 0] ** 2 + median_flow[..., 1] ** 2)
    p99 = float(np.percentile(med_mag, 99.0)) if med_mag.size else 0.0
    clip_max = float(min(max(p99, 0.0), max(flow_max_mag, 0.0)))
    clipped = clip_flow_magnitude(median_flow, clip_max)
    smoothed = cv2.GaussianBlur(clipped, (0, 0), sigmaX=FLOW_SMOOTH_SIGMA, sigmaY=FLOW_SMOOTH_SIGMA)
    centered = stack - stack.mean(axis=0, keepdims=True)
    var_scalar = float(np.mean(centered[..., 0] ** 2 + centered[..., 1] ** 2))
    return smoothed.astype(np.float32), var_scalar, p99, clip_max


def resize_flow_to_image(flow: np.ndarray, h: int, w: int) -> np.ndarray:
    fh, fw = flow.shape[:2]
    resized = cv2.resize(flow, (w, h), interpolation=cv2.INTER_CUBIC)
    resized[..., 0] *= float(w) / float(fw)
    resized[..., 1] *= float(h) / float(fh)
    return resized.astype(np.float32)


def apply_residual_flow(image: np.ndarray, flow: np.ndarray, lam: float) -> np.ndarray:
    if lam <= 0.0:
        return image.copy()
    h, w = image.shape[:2]
    f = resize_flow_to_image(flow, h, w) * float(lam)
    ys, xs = np.mgrid[0:h, 0:w].astype(np.float32)
    map_x = xs - f[..., 0]
    map_y = ys - f[..., 1]
    return cv2.remap(image, map_x, map_y, interpolation=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT)


def compute_black_border_ratio(image: np.ndarray, threshold: int = 2) -> float:
    return float(np.all(image <= threshold, axis=2).mean())


def residual_warp_risk(flow: np.ndarray, lam: float) -> float:
    if lam <= 0.0:
        return 0.0
    mag = np.sqrt(flow[..., 0] ** 2 + flow[..., 1] ** 2)
    return float(abs(lam) * np.percentile(mag, 99.0)) if mag.size else 0.0


def to_gray(image: np.ndarray) -> np.ndarray:
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) if image.ndim == 3 else image


def canny_union(gray: np.ndarray) -> np.ndarray:
    e1 = cv2.Canny(gray, 50, 150)
    e2 = cv2.Canny(gray, 80, 160)
    e3 = cv2.Canny(gray, 110, 220)
    return cv2.bitwise_or(cv2.bitwise_or(e1, e2), e3)


def edge_similarity(pred: np.ndarray, gt: np.ndarray) -> float:
    p = canny_union(to_gray(pred)) > 0
    g = canny_union(to_gray(gt)) > 0
    tp = int(np.logical_and(p, g).sum())
    fp = int(np.logical_and(p, ~g).sum())
    fn = int(np.logical_and(~p, g).sum())
    if tp == 0 and fp == 0 and fn == 0:
        return 1.0
    if tp == 0:
        return 0.0
    return float((2.0 * tp) / (2.0 * tp + fp + fn))


def line_hist(gray: np.ndarray, bins: int = 18) -> np.ndarray:
    angles = []
    if hasattr(cv2, "createLineSegmentDetector"):
        lsd = cv2.createLineSegmentDetector()
        lines = lsd.detect(gray)[0]
        if lines is not None:
            for item in lines:
                x1, y1, x2, y2 = item[0]
                angle = np.degrees(np.arctan2(float(y2 - y1), float(x2 - x1)))
                angles.append((angle + 180.0) % 180.0)
    if not angles:
        edges = canny_union(gray)
        min_line = max(8, min(gray.shape[0], gray.shape[1]) // 6)
        lines = cv2.HoughLinesP(edges, 1, np.pi / 180.0, 30, minLineLength=min_line, maxLineGap=6)
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                angle = np.degrees(np.arctan2(float(y2 - y1), float(x2 - x1)))
                angles.append((angle + 180.0) % 180.0)
    if not angles:
        return np.zeros(bins, dtype=np.float32)
    hist, _ = np.histogram(np.array(angles), bins=bins, range=(0.0, 180.0))
    hist = hist.astype(np.float32)
    total = float(hist.sum())
    return hist / total if total > 0 else np.zeros(bins, dtype=np.float32)


def line_orientation_loss(pred: np.ndarray, gt: np.ndarray) -> float:
    p = line_hist(to_gray(pred))
    g = line_hist(to_gray(gt))
    ps, gs = float(p.sum()), float(g.sum())
    if ps == 0.0 and gs == 0.0:
        return 0.0
    if ps == 0.0 or gs == 0.0:
        return 1.0
    return float(0.5 * np.abs(p - g).sum())


def grad_hist(gray: np.ndarray, bins: int = 36) -> np.ndarray:
    gx = cv2.Scharr(gray, cv2.CV_32F, 1, 0)
    gy = cv2.Scharr(gray, cv2.CV_32F, 0, 1)
    mag, ang = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    mag = mag.astype(np.float32)
    threshold = float(np.percentile(mag, 60.0)) if mag.size else 0.0
    mask = mag > max(threshold, 1e-6)
    if not np.any(mask):
        return np.zeros(bins, dtype=np.float32)
    hist, _ = np.histogram(ang[mask], bins=bins, range=(0.0, 360.0), weights=mag[mask])
    hist = hist.astype(np.float32)
    total = float(hist.sum())
    return hist / total if total > 0 else np.zeros(bins, dtype=np.float32)


def grad_orientation_loss(pred: np.ndarray, gt: np.ndarray) -> float:
    p = grad_hist(to_gray(pred))
    g = grad_hist(to_gray(gt))
    pn, gn = float(np.linalg.norm(p)), float(np.linalg.norm(g))
    if pn == 0.0 and gn == 0.0:
        return 0.0
    if pn == 0.0 or gn == 0.0:
        return 1.0
    cosine = float(np.dot(p, g) / (pn * gn + 1e-8))
    return float(np.clip(1.0 - cosine, 0.0, 1.0))


def ssim_score(pred: np.ndarray, gt: np.ndarray) -> float:
    x = to_gray(pred).astype(np.float32)
    y = to_gray(gt).astype(np.float32)
    if x.shape != y.shape:
        y = cv2.resize(y, (x.shape[1], x.shape[0]))
    mu_x = float(x.mean())
    mu_y = float(y.mean())
    sigma_x = float(((x - mu_x) ** 2).mean())
    sigma_y = float(((y - mu_y) ** 2).mean())
    sigma_xy = float(((x - mu_x) * (y - mu_y)).mean())
    c1 = (0.01 * 255.0) ** 2
    c2 = (0.03 * 255.0) ** 2
    denom = (mu_x**2 + mu_y**2 + c1) * (sigma_x + sigma_y + c2)
    if denom <= 0.0:
        return 0.0
    return float(np.clip(((2.0 * mu_x * mu_y + c1) * (2.0 * sigma_xy + c2)) / denom, 0.0, 1.0))


def normalized_mae(pred: np.ndarray, gt: np.ndarray) -> float:
    if pred.shape != gt.shape:
        gt = cv2.resize(gt, (pred.shape[1], pred.shape[0]))
    return float(np.mean(np.abs(pred.astype(np.float32) - gt.astype(np.float32))) / 255.0)


def competition_lite_loss(pred: np.ndarray, gt: np.ndarray) -> float:
    edge = edge_similarity(pred, gt)
    line = line_orientation_loss(pred, gt)
    grad = grad_orientation_loss(pred, gt)
    ssim = ssim_score(pred, gt)
    mae = normalized_mae(pred, gt)
    return float(
        0.40 * (1.0 - edge)
        + 0.22 * line
        + 0.18 * grad
        + 0.15 * (1.0 - ssim)
        + 0.05 * mae
    )


def should_accept(proxy_gain: float, border_ratio: float, warp_risk: float, flow_var: float):
    if flow_var > FLOW_STABILITY_VAR_MAX:
        return False, "reject_unstable_flow"
    if proxy_gain < ACCEPT_MIN_PROXY_GAIN:
        return False, "reject_low_proxy_gain"
    if border_ratio > BORDER_RATIO_MAX:
        return False, "reject_border_ratio"
    if warp_risk > WARP_RISK_MAX:
        return False, "reject_warp_risk"
    return True, "accepted"


def parse_dim_key(dim_key: str):
    w, h = dim_key.lower().split("x", 1)
    return int(w), int(h)


def discover_data_dirs():
    roots = [
        Path("/kaggle/input/automatic-lens-correction"),
        Path("/kaggle/input/competitions/automatic-lens-correction"),
        Path("/kaggle/input"),
    ]
    train_dir = None
    test_dir = None

    for root in roots:
        if not root.exists():
            continue
        for path in [root] + [p for p in root.rglob("*") if p.is_dir()]:
            if train_dir is None and any(path.glob("*_original.jpg")):
                train_dir = path
            if test_dir is None:
                jpg = list(path.glob("*.jpg"))
                if jpg and not any(path.glob("*_original.jpg")) and not any(path.glob("*_generated.jpg")):
                    test_dir = path
            if train_dir is not None and test_dir is not None:
                break
        if train_dir is not None and test_dir is not None:
            break

    if train_dir is None or test_dir is None:
        raise RuntimeError(f"Failed to discover data dirs. train={train_dir} test={test_dir}")
    return train_dir, test_dir


def load_table():
    env_path = os.getenv("DIM_TABLE_PATH", "").strip()
    candidates = []
    if env_path:
        candidates.append(Path(env_path))
    candidates.extend([
        Path("/kaggle/input/calibguard-dim-table/calibguard_dim_table.json"),
        Path("/kaggle/working/calibguard_dim_table.json"),
    ])
    for path in candidates:
        if path.exists():
            return json.loads(path.read_text()), str(path)
    return DEFAULT_DIM_TABLE, "embedded_defaults"


def load_train_index(train_dir: Path):
    pairs_by_dim = defaultdict(list)
    originals = sorted([p for p in train_dir.glob("*_original.jpg")])
    for idx, orig in enumerate(originals, start=1):
        gen = train_dir / orig.name.replace("_original.jpg", "_generated.jpg")
        if not gen.exists():
            continue
        img = cv2.imread(str(orig))
        if img is None:
            continue
        h, w = img.shape[:2]
        pairs_by_dim[f"{w}x{h}"].append((orig, gen))
        if idx % 4000 == 0:
            print(f"Indexed {idx}/{len(originals)} train originals")
    return pairs_by_dim


def evaluate_lambdas(holdout_pairs, k1, k2, model_type, flow_map):
    stats = {}
    for lam in sorted(set([0.0] + LAMBDA_GRID)):
        losses = []
        borders = []
        for orig, target in holdout_pairs:
            base = apply_base_undistortion(orig, k1, k2, model_type)
            pred = apply_residual_flow(base, flow_map, lam)
            losses.append(competition_lite_loss(pred, target))
            borders.append(compute_black_border_ratio(pred))
        if losses:
            stats[lam] = (float(np.mean(losses)), float(np.mean(borders)))
        else:
            stats[lam] = (float("inf"), 1.0)

    base_loss = stats[0.0][0]
    best_lam = min(stats, key=lambda x: stats[x][0])
    best_loss, best_border = stats[best_lam]
    return best_lam, base_loss, best_loss, best_border


def calibrate_dim_model(dim_key, pairs, table):
    w, h = parse_dim_key(dim_key)
    parent = classify_parent(h, w)
    k1, k2, coeff_source = choose_coeffs(table, dim_key, parent, PROFILE)
    model_type = choose_model_type(table, dim_key)

    sample = list(pairs)
    random.shuffle(sample)
    sample = sample[: min(len(sample), TRAIN_SAMPLE_PER_DIM)]

    if len(sample) < 4:
        return DimModel(dim_key, len(pairs), parent, k1, k2, model_type, 0.0, False, "reject_not_enough_samples", 0.0, float("inf"), float("inf"), 1.0, float("inf"), 0.0, 0.0, 0.0, None)

    split_idx = max(1, int(round(len(sample) * (1.0 - HOLDOUT_RATIO))))
    split_idx = min(split_idx, len(sample) - 1)
    train_subset = sample[:split_idx]
    holdout_subset = sample[split_idx:]

    flow_stack = []
    for orig_path, gen_path in train_subset:
        orig = cv2.imread(str(orig_path))
        target = cv2.imread(str(gen_path))
        if orig is None or target is None:
            continue
        base = apply_base_undistortion(orig, k1, k2, model_type)
        if FLOW_WORK_SCALE != 1.0:
            base = cv2.resize(base, None, fx=FLOW_WORK_SCALE, fy=FLOW_WORK_SCALE, interpolation=cv2.INTER_AREA)
            target = cv2.resize(target, (base.shape[1], base.shape[0]), interpolation=cv2.INTER_AREA)
        flow_stack.append(compute_dense_flow(base, target))

    if not flow_stack:
        return DimModel(dim_key, len(pairs), parent, k1, k2, model_type, 0.0, False, "reject_no_flow_pairs", 0.0, float("inf"), float("inf"), 1.0, float("inf"), 0.0, 0.0, 0.0, None)

    flow_map, flow_var, flow_p99, flow_clip_max = aggregate_residual_flow(flow_stack, FLOW_MAX_MAG)

    holdout_pairs = []
    for orig_path, gen_path in holdout_subset:
        orig = cv2.imread(str(orig_path))
        target = cv2.imread(str(gen_path))
        if orig is None or target is None:
            continue
        holdout_pairs.append((orig, target))

    best_lam, base_loss, best_loss, best_border = evaluate_lambdas(holdout_pairs, k1, k2, model_type, flow_map)
    proxy_gain = float(base_loss - best_loss)
    warp = residual_warp_risk(flow_map, best_lam)
    accepted, reason = should_accept(proxy_gain, best_border, warp, flow_var)
    if not accepted:
        best_lam = 0.0

    return DimModel(
        dim_key=dim_key,
        support=len(pairs),
        parent_class=parent,
        k1=k1,
        k2=k2,
        model_type=model_type,
        lambda_value=float(best_lam),
        accepted=accepted,
        reason=reason,
        proxy_gain=float(proxy_gain),
        base_loss=float(base_loss),
        best_loss=float(best_loss),
        border_ratio=float(best_border),
        flow_var=float(flow_var),
        flow_p99=float(flow_p99),
        flow_clip_max=float(flow_clip_max),
        warp_risk=float(warp),
        flow_map=flow_map if accepted else None,
    )


def zip_dir(input_dir: Path, zip_path: Path, names: list[str]):
    with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_STORED) as zf:
        for name in names:
            path = input_dir / name
            if path.exists():
                zf.write(path, name)


def main():
    t0 = time.time()
    train_dir, test_dir = discover_data_dirs()
    table, table_source = load_table()

    print(f"Train dir: {train_dir}")
    print(f"Test dir: {test_dir}")
    print(f"Table source: {table_source}")

    train_index = load_train_index(train_dir)

    dim_entries = table.get("dimensions", {}) if isinstance(table.get("dimensions"), dict) else {}
    if not dim_entries:
        # if table has no exact dims, infer from observed train dims using parent defaults
        for dim_key, pairs in train_index.items():
            dim_entries[dim_key] = {"support": len(pairs)}

    eligible_dims = []
    for dim_key, entry in dim_entries.items():
        if not isinstance(entry, dict):
            continue
        support = int(entry.get("support", 0))
        if support < MIN_SUPPORT:
            continue
        if dim_key not in train_index:
            continue
        eligible_dims.append(dim_key)
    eligible_dims = sorted(eligible_dims)

    print(f"Eligible dimensions: {len(eligible_dims)}")

    models = {}
    for idx, dim_key in enumerate(eligible_dims, start=1):
        model = calibrate_dim_model(dim_key, train_index[dim_key], table)
        models[dim_key] = model
        print(
            f"[{idx}/{len(eligible_dims)}] {dim_key} support={model.support} "
            f"lambda={model.lambda_value:.3f} gain={model.proxy_gain:.5f} "
            f"accepted={model.accepted} ({model.reason})"
        )

    test_files = sorted([p.name for p in test_dir.glob("*.jpg")])

    source_counts = Counter()
    parent_counts = Counter()

    for idx, name in enumerate(test_files, start=1):
        image = cv2.imread(str(test_dir / name))
        if image is None:
            continue
        h, w = image.shape[:2]
        dim_key = f"{w}x{h}"
        parent = classify_parent(h, w)
        parent_counts[parent] += 1

        k1, k2, _source = choose_coeffs(table, dim_key, parent, PROFILE)
        model_type = choose_model_type(table, dim_key)
        base = apply_base_undistortion(image, k1, k2, model_type)

        cv2.imwrite(str(BASE_DIR / name), base, [cv2.IMWRITE_JPEG_QUALITY, JPEG_QUALITY])

        dm = models.get(dim_key)
        if dm is not None and dm.accepted and dm.flow_map is not None:
            pred = apply_residual_flow(base, dm.flow_map, dm.lambda_value)
            source_counts["base_plus_residual"] += 1
        else:
            pred = base
            source_counts["base_only"] += 1

        cv2.imwrite(str(CANDIDATE_DIR / name), pred, [cv2.IMWRITE_JPEG_QUALITY, JPEG_QUALITY])

        if idx % 100 == 0 or idx == len(test_files):
            print(f"Rendered {idx}/{len(test_files)}")

    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    candidate_zip = WORK_ROOT / f"submission_flow_residual_dim_v1_{ts}.zip"
    base_zip = WORK_ROOT / f"submission_flow_residual_dim_base_{ts}.zip"
    summary_json = WORK_ROOT / f"run_summary_{ts}.json"

    zip_dir(CANDIDATE_DIR, candidate_zip, test_files)
    zip_dir(BASE_DIR, base_zip, test_files)

    dim_summary = {}
    for dim_key, model in models.items():
        dim_summary[dim_key] = {
            "support": model.support,
            "parent_class": model.parent_class,
            "base_coeffs": {"k1": model.k1, "k2": model.k2, "model_type": model.model_type},
            "lambda": model.lambda_value,
            "proxy_gain": model.proxy_gain,
            "base_loss": model.base_loss,
            "best_loss": model.best_loss,
            "border_ratio": model.border_ratio,
            "flow_var": model.flow_var,
            "flow_p99": model.flow_p99,
            "flow_clip_max": model.flow_clip_max,
            "warp_risk": model.warp_risk,
            "accepted": model.accepted,
            "reason": model.reason,
        }

    summary = {
        "method": "flow_residual_dim_v1",
        "timestamp_utc": datetime.utcnow().isoformat(),
        "train_dir": str(train_dir),
        "test_dir": str(test_dir),
        "table_source": table_source,
        "config": {
            "min_support": MIN_SUPPORT,
            "train_sample_per_dim": TRAIN_SAMPLE_PER_DIM,
            "holdout_ratio": HOLDOUT_RATIO,
            "flow_max_mag": FLOW_MAX_MAG,
            "lambda_grid": sorted(set([0.0] + LAMBDA_GRID)),
            "accept_min_proxy_gain": ACCEPT_MIN_PROXY_GAIN,
            "profile": PROFILE,
            "alpha_forced": 0.0,
        },
        "thresholds": {
            "border_ratio_max": BORDER_RATIO_MAX,
            "warp_risk_max": WARP_RISK_MAX,
            "flow_stability_var_max": FLOW_STABILITY_VAR_MAX,
        },
        "counts": {
            "test_images": len(test_files),
            "eligible_dimensions": len(eligible_dims),
            "accepted_dimensions": int(sum(1 for m in models.values() if m.accepted)),
            "source_counts": dict(source_counts),
            "parent_counts": dict(parent_counts),
        },
        "outputs": {
            "candidate_zip": str(candidate_zip),
            "base_zip": str(base_zip),
            "recommended_submission_zip": str(candidate_zip),
        },
        "dimensions": dim_summary,
        "elapsed_seconds": float(time.time() - t0),
    }

    summary_json.write_text(json.dumps(summary, indent=2), encoding="utf-8")

    print("\n=== COMPLETE ===")
    print(f"Candidate zip: {candidate_zip}")
    print(f"Base zip: {base_zip}")
    print(f"Run summary: {summary_json}")


main()
